In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean
from pyproj import Proj

In [ ]:
## YYYY-MM-DD
start_date = np.datetime64('2019-01-01')
end_date   = np.datetime64('2019-01-07')
num_days = (end_date - start_date).tolist().days

# variable to load
VAR = 'CHL'

# algorithm
ALG = 'chl_ocx'

# Binning period
BIN = 'DAY'  # DAY, 8D, MO, R32

# Spatial resolution
SRES = '4km'   # 4km, 9km

dap_urls = []

url_base = "https://oceandata.sci.gsfc.nasa.gov:443/opendap/MODISA/L3SMI/"

for ii in range(num_days):
    curr_date = start_date + ii
    
    curr_year = curr_date.tolist().year
    ref_date = np.datetime64('{0:d}-01-01'.format(curr_year))
    
    day_num = 1 + (curr_date - ref_date).tolist().days
    
    file_url = url_base + \
                '{0:d}/{1:03d}/A{0:d}{1:03d}'.format(curr_year, day_num) + \
                '.L3m_{0}_{1}_{2}_{3}'.format(BIN, VAR, ALG, SRES) + \
                '.nc'
    
    dap_urls += [file_url]
    
print('dap_urls containts {0:d} urls for {1} data.'.format(len(dap_urls), VAR))

In [ ]:
single_set = xr.open_dataset(dap_urls[0])

In [ ]:
lon_lb = -130
lon_ub = -110

lat_lb = 25
lat_ub = 40

In [ ]:
def preprocess_set(dset, time):
    return dset.sel(lon=slice(lon_lb,lon_ub),lat=slice(lat_ub,lat_lb))

In [ ]:
data_sets = [preprocess_set(xr.open_dataset(url), ind) \
             for (url,ind) \
             in zip(dap_urls, np.arange(num_days))]

In [ ]:
merged = xr.concat(data_sets, 'time')

In [ ]:
merged.chl_ocx.mean(dim=['lat', 'lon']).plot()
plt.gca().set_yscale('log')

In [ ]:
mean_chl = merged.chl_ocx.mean(dim='time').data
std_chl  = merged.chl_ocx.std( dim='time').data

lon = merged.lon.data
lat = merged.lat.data

In [ ]:
meridians = np.round(np.linspace(lon.min(), lon.max(), 5))
parallels = np.round(np.linspace(lat.min(), lat.max(), 5))

In [ ]:
proj = Proj(proj='wag7', lon_0 = lon.mean(), lat_0 = lat.mean())

In [ ]:
LON, LAT = np.meshgrid(lon, lat)

Xp, Yp = proj(LON, LAT, inverse=False)

In [ ]:
def AddParallels_and_Meridians(
        ax, proj,
        parallels, meridians,
        latitude, longitude,
        label_parallels = True,
        label_meridians = True):

    for mer in meridians:
        xs, ys = proj(mer*np.ones(latitude.shape), latitude, inverse=False)
        ax.plot(xs, ys, '--g', linewidth=1)
    for par in parallels:
        xs, ys = proj(longitude, par*np.ones(longitude.shape), inverse=False)
        ax.plot(xs, ys, '--g', linewidth=1)

    xs, ys = proj(meridians, latitude.min()*np.ones(meridians.shape), inverse=False)
    ax.set_xticks(xs)
    if label_meridians:
        ax.set_xticklabels(['{0:d}'.format(int(mer)) for mer in meridians])
    else:
        ax.set_xticklabels([])

    xs, ys = proj(longitude.min()*np.ones(parallels.shape), parallels, inverse=False)
    ax.set_yticks(ys)
    if label_parallels:
        ax.set_yticklabels(['{0:d}'.format(int(par)) for par in parallels])
    else:
        ax.set_yticklabels([])

In [ ]:
fig, axes = plt.subplots(1, 3, sharex=True, figsize=(20,4))

q0 = axes[0].pcolormesh(Xp, Yp, np.log10(mean_chl), vmin=0, vmax=0.2, cmap='cmo.algae')
q1 = axes[1].pcolormesh(Xp, Yp, std_chl, vmin=0, vmax=0.4, cmap='cmo.algae')
q2 = axes[2].pcolormesh(Xp, Yp, std_chl / (mean_chl + 1e-5), vmin=0, vmax=1, cmap='cmo.algae')

plt.colorbar(q0, ax=axes[0])
plt.colorbar(q1, ax=axes[1])
plt.colorbar(q2, ax=axes[2])

axes[0].set_title('$log_{10}(\mu_t(\mathrm{CHL}_{\mathrm{ocx}})$')
axes[1].set_title('$\sigma_t(\mathrm{CHL}_{\mathrm{ocx}})$')
axes[2].set_title('$\sigma_t / \mu_t$')

for ax in axes:

    ax.set_xticks([])
    ax.set_yticks([])
    
    AddParallels_and_Meridians(ax, proj, parallels, meridians, lat, lon)
    
    ax.set_aspect('equal')